# INSTALLING DEPENDENCIES AND IMPORTING THEM

In [1]:
import mediapipe as mp
import cv2 

In [2]:
mp_drawing=mp.solutions.drawing_utils
mp_holistic=mp.solutions.holistic

# WEBCAME VIDEO FOR LENGTH OF CO-ORDINATES

In [7]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable=False #preventing to show this on webcam
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        image.flags.writeable=True
        
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# EXPORTING CO-ORDINATES TO CSV FILE

In [4]:
import csv
import numpy as np

In [12]:
num_coords=len(results.pose_landmarks.landmark)+len(results.face_landmarks.landmark)

In [13]:
landmarks=['class']
for val in range(1,num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [14]:
with open('detection.csv',mode='w',newline='')as f:
    csv_writer=csv.writer(f,delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

# PERFORMING OUR DETECTIONS

In [9]:
class_name="rukooh"

In [10]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable=False #preventing to show this on webcam
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable=True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        
        #Exporting Co ordinates
        pose=results.pose_landmarks.landmark
        pose_row=list(np.array([[landmarks.x,landmarks.y,landmarks.z,landmarks.visibility]for landmarks in pose]).flatten())
        
        face=results.face_landmarks.landmark
        face_row=list(np.array([[landmarks.x,landmarks.y,landmarks.z,landmarks.visibility]for landmarks in face]).flatten())
        
        #row making
        row=pose_row+face_row
        row.insert(0,class_name)
        
        with open('detection.csv',mode='a',newline='')as f:
            csv_writer=csv.writer(f,delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(row)
        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

AttributeError: 'NoneType' object has no attribute 'landmark'

# Training Model

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df=pd.read_csv('detection.csv')

In [3]:
df = df[df.columns[~df.isnull().all()]]

In [4]:
x=df.drop('class',axis=1)
y=df['class']

In [5]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=1234)

In [6]:
x_train

,x1,y1,z1,v1,x2,y2,z2,v2,x3,y3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
129,0.502693,0.370511,-0.816438,0.999952,0.510474,0.355221,-0.816423,0.999861,0.515339,0.353583,...,0.009927,0.851622,0.554313,0.875758,-0.173699,0.978127,0.472903,0.884036,-0.168844,0.976643
35,0.504352,0.220616,-0.277896,0.999113,0.513196,0.201275,-0.255267,0.997810,0.518214,0.201683,...,0.245299,0.579870,0.540981,1.061053,-0.006361,0.773731,0.458235,1.055096,0.071297,0.816370
20,0.494637,0.545608,-0.666845,0.999831,0.502750,0.527526,-0.667335,0.999555,0.507565,0.525929,...,0.174259,0.144141,0.535025,1.140810,0.015632,0.208113,0.437184,1.126759,0.041271,0.220144
136,0.503497,0.363081,-0.818383,0.999952,0.511054,0.347257,-0.817023,0.999871,0.515725,0.345862,...,0.023397,0.815479,0.557002,0.875396,-0.193331,0.961806,0.466951,0.883748,-0.145943,0.955539
104,0.516338,0.199090,-0.207180,0.997650,0.522058,0.178682,-0.189676,0.993809,0.525592,0.178075,...,0.215831,0.835732,0.542315,0.861989,0.040012,0.972062,0.471608,0.864480,0.067796,0.969447
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,0.476121,0.455111,-0.370952,0.999974,0.485686,0.435504,-0.358217,0.999961,0.491590,0.435708,...,0.147878,0.064379,0.493043,0.856772,0.203421,0.037478,0.450977,0.845587,0.184336,0.051492
116,0.503971,0.361001,-0.830732,0.999935,0.511114,0.346669,-0.828728,0.999802,0.515583,0.345143,...,0.037888,0.844090,0.553712,0.873199,-0.157852,0.978021,0.470549,0.877588,-0.136647,0.975749
53,0.522900,0.373342,-1.272473,0.999999,0.539907,0.342655,-1.254972,0.999997,0.548180,0.341875,...,0.986194,0.155800,0.558394,0.745693,0.922659,0.407121,0.482087,0.735188,0.964557,0.428441
38,0.503898,0.222388,-0.257756,0.998144,0.511469,0.204048,-0.233599,0.996497,0.516321,0.204470,...,0.202420,0.599477,0.546941,1.062232,-0.014654,0.793932,0.458368,1.058989,0.017842,0.839006


# Fitting algorithm in model

In [7]:
from sklearn.pipeline import make_pipeline

In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression,RidgeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier

In [9]:
pipelines={
    'lr':make_pipeline(StandardScaler(),LogisticRegression()),
    'rc':make_pipeline(StandardScaler(),RidgeClassifier()),
    'rfc':make_pipeline(StandardScaler(),RandomForestClassifier()),
    'gbc':make_pipeline(StandardScaler(),GradientBoostingClassifier()),
}

In [10]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(x_train, y_train)
    fit_models[algo] = model

In [12]:
x_test = x_test.fillna(x_train.mean())
x_test

,x1,y1,z1,v1,x2,y2,z2,v2,x3,y3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
46,0.499656,0.978009,-0.621408,0.999324,0.518260,0.942374,-0.649289,0.999685,0.534070,0.936957,...,-0.005878,0.0,0.406621,0.372553,-0.001939,0.0,0.407727,0.370844,-0.002058,0.0
57,0.467677,1.017901,-0.316981,0.999406,0.475528,1.031896,-0.362747,0.999755,0.481240,1.004771,...,-0.005878,0.0,0.406621,0.372553,-0.001939,0.0,0.407727,0.370844,-0.002058,0.0
33,0.477727,0.960362,-0.258159,0.999718,0.494918,0.974285,-0.320807,0.999850,0.507511,0.973777,...,-0.005878,0.0,0.406621,0.372553,-0.001939,0.0,0.407727,0.370844,-0.002058,0.0
36,0.477601,0.962562,-0.281356,0.999589,0.496179,0.972707,-0.356246,0.999786,0.508981,0.972008,...,-0.005878,0.0,0.406621,0.372553,-0.001939,0.0,0.407727,0.370844,-0.002058,0.0
6,0.385423,0.404869,-0.802578,0.999990,0.392710,0.386668,-0.798266,0.999975,0.397085,0.385336,...,-0.005933,0.0,0.409347,0.369147,-0.001152,0.0,0.410361,0.367419,-0.001205,0.0
25,0.494978,0.912415,-0.553456,0.999988,0.517770,0.898197,-0.596208,0.999994,0.524588,0.893882,...,-0.005878,0.0,0.406621,0.372553,-0.001939,0.0,0.407727,0.370844,-0.002058,0.0
63,0.485518,0.923402,-0.951093,0.998609,0.498222,0.905304,-0.971618,0.999267,0.503743,0.902202,...,-0.005878,0.0,0.406621,0.372553,-0.001939,0.0,0.407727,0.370844,-0.002058,0.0
66,0.445127,0.814839,-0.886262,0.997487,0.461141,0.780196,-0.905260,0.998595,0.475481,0.777562,...,-0.005878,0.0,0.406621,0.372553,-0.001939,0.0,0.407727,0.370844,-0.002058,0.0
8,0.387587,0.404981,-0.814530,0.999990,0.394851,0.386751,-0.810919,0.999975,0.399093,0.385453,...,-0.005679,0.0,0.409633,0.371812,-0.001691,0.0,0.410775,0.370089,-0.001784,0.0
20,0.383866,0.406579,-0.824854,0.999991,0.389301,0.389239,-0.821529,0.999978,0.393969,0.387876,...,-0.005756,0.0,0.407605,0.374178,-0.001519,0.0,0.408729,0.372741,-0.001661,0.0


In [11]:
fit_models['rfc'].predict(x_test)

array(['julsa', 'julsa', 'rukooh', 'rukooh', 'julsa', 'qiam', 'julsa',
       'Sajdah', 'qiam', 'qiam', 'Sajdah', 'qiam', 'qiam', 'julsa',
       'julsa', 'qiam', 'julsa', 'Sajdah', 'julsa', 'Sajdah', 'Sajdah',
       'Sajdah', 'rukooh', 'julsa', 'rukooh', 'Sajdah', 'rukooh',
       'Sajdah', 'julsa', 'Sajdah', 'qiam', 'rukooh', 'julsa', 'julsa',
       'qiam', 'Sajdah', 'qiam', 'Sajdah', 'Sajdah', 'rukooh', 'qiam',
       'qiam', 'qiam', 'qiam', 'qiam', 'julsa', 'rukooh', 'julsa',
       'Sajdah', 'julsa', 'rukooh', 'rukooh', 'rukooh', 'Sajdah'],
      dtype='<U6')

In [12]:
y_test

157     julsa
145     julsa
125    rukooh
131    rukooh
146     julsa
95       qiam
165     julsa
74     Sajdah
91       qiam
39       qiam
77     Sajdah
102      qiam
31       qiam
172     julsa
167     julsa
40       qiam
9       julsa
54     Sajdah
142     julsa
79     Sajdah
49     Sajdah
43     Sajdah
123    rukooh
177     julsa
122    rukooh
63     Sajdah
25     rukooh
59     Sajdah
6       julsa
51     Sajdah
29       qiam
121    rukooh
149     julsa
174     julsa
111      qiam
48     Sajdah
105      qiam
65     Sajdah
58     Sajdah
27     rukooh
88       qiam
92       qiam
32       qiam
99       qiam
108      qiam
160     julsa
133    rukooh
11      julsa
61     Sajdah
155     julsa
132    rukooh
17     rukooh
119    rukooh
83     Sajdah
Name: class, dtype: object

In [13]:
from sklearn.metrics import accuracy_score
import pickle

In [14]:
for algo,model in fit_models.items():
    yhat=model.predict(x_test)
    print(algo,accuracy_score(y_test,yhat))

lr 1.0
rc 1.0
rfc 1.0
gbc 1.0


In [15]:
with open ('body_language.pkl','wb')as f:
    pickle.dump(fit_models['rfc'],f)

# TESTING MODEL

In [1]:
import mediapipe as mp
import cv2 
import csv
import numpy as np
import pandas as pd

In [2]:
mp_drawing=mp.solutions.drawing_utils
mp_holistic=mp.solutions.holistic

In [3]:
import pickle
with open ('body_language.pkl','rb')as f:
    model=pickle.load(f)

In [ ]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable=False #preventing to show this on webcam
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable=True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        
        #Exporting Co ordinates
        pose=results.pose_landmarks.landmark
        pose_row=list(np.array([[landmarks.x,landmarks.y,landmarks.z,landmarks.visibility]for landmarks in pose]).flatten())
        

        
        #row making
        row=pose_row
        X=pd.DataFrame([row])
        body_language_class=model.predict(X)[0] ##class_name
        body_language_prob=model.predict_proba(X)[0]
       
    
        
        
            # Grab ear coords
        coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
        
            
        cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
        
        if round(body_language_prob[np.argmax(body_language_prob)],2)< 0.5:
               
                cv2.putText(image, "NONE", coords, 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
               
        else:
                cv2.putText(image, body_language_class, coords, 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            # Get status box
        cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
        
            
            # Display Class
        
        cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        
        if round(body_language_prob[np.argmax(body_language_prob)],2)< 0.5:
               
                cv2.putText(image, "NONE", coords, 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
               
        else:
                cv2.putText(image, body_language_class.split(' ')[0]
                , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
        cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            
            
        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()